In [1]:
import geotrellis.proj4.CRS
import geotrellis.raster.{ArrayTile, DoubleArrayTile, Tile}
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.spark.io.hadoop._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

In [4]:
    val band_count = 1;
    var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
    var num_cols_rows :(Int, Int) = (0, 0)
    var band_RDD: RDD[Array[Double]] = sc.emptyRDD
    var band_vec: RDD[Vector] = sc.emptyRDD
    var band0: RDD[(Long, Double)] = sc.emptyRDD
    var band0_index: RDD[Long] = sc.emptyRDD
    val pattern: String = "2.tif"
    var filepath: String = ""
    if (band_count == 1) {
      //Single band GeoTiff
      filepath = "hdfs:///user/hadoop/spring-index/LastFreeze/1980.tif"
    } else {
      //Multi band GeoTiff
      filepath = "hdfs:///user/hadoop/spring-index/BloomFinal/1980.tif"
    }

    if (band_count == 1) {
      //Lets load a Singleband GeoTiff and return RDD just with the tiles.
      //Since it is a single GeoTiff, it will be a RDD with a tile.
      val tiles_RDD = sc.hadoopGeoTiffRDD(filepath).values
      val bands_RDD = tiles_RDD.map(m => m.toArrayDouble())

      val extents_withIndex = sc.hadoopGeoTiffRDD(filepath).keys.zipWithIndex().map{case (e,v) => (v,e)}
      projected_extent = (extents_withIndex.filter(m => m._1 == 0).values.collect())(0)

      val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
      //num_cols_rows = (tiles_withIndex.lookup(0).apply(0).cols, tiles_withIndex.lookup(0).apply(0).rows)
      val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
      num_cols_rows = (tile0.cols,tile0.rows)
        
      println(tile0.cellType)

      //Get Index for Cells
      val bands_withIndex = bands_RDD.zipWithIndex().map { case (e, v) => (v, e) }
      //band0_index = bands_withIndex.lookup(0).apply(0).zipWithIndex.filter{ case (v, i) => !v.isNaN }.map { case (v, i) => (i) }
      //val band0_index = bands_withIndex.lookup(0).apply(0).zipWithIndex.filter(m => !m._1.isNaN).take(sample).map { case (v, i) => (i) }
      band0_index = bands_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => !m._1.isNaN).map { case (v, i) => (i) }

      //Get Array[Double] of a Title to later store the cluster ids.
      //band0 = sc.parallelize(bands_withIndex.lookup(0).take(1)).flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}
      band0 = bands_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

      //Lets filter out NaN
      band_RDD = bands_RDD.map(m => m.filter(!_.isNaN))
    } else {
      //Lets load a Multiband GeoTiff and return RDD just with the tiles.
      //Since it is a multi-band GeoTiff, we will take band 4
      val tiles_RDD = sc.hadoopMultibandGeoTiffRDD(filepath).values
      val bands_RDD = tiles_RDD.map(m => m.band(3).toArrayDouble())

      val extents_withIndex = sc.hadoopGeoTiffRDD(filepath).keys.zipWithIndex().map{case (e,v) => (v,e)}
      projected_extent = (extents_withIndex.filter(m => m._1 == 0).values.collect())(0)

      val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
      val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
      num_cols_rows = (tile0.cols,tile0.rows)

      //Get Index for Cells
      val bands_withIndex = bands_RDD.zipWithIndex().map { case (e, v) => (v, e) }
      //band0_index = bands_withIndex.lookup(0).apply(0).zipWithIndex.filter { case (v, i) => !v.isNaN }.take(sample).map { case (v, i) => (i) }
      band0_index = bands_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => !m._1.isNaN).map { case (v, i) => (i) }

      //Get Array[Double] of a Title to later store the cluster ids.
      band0 = bands_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

      //Let's filter out NaN
      band_RDD = bands_RDD.map(m => m.filter(v => !v.isNaN))
    }

uint8raw


band_count = 1
projected_extent = ProjectedExtent(Extent(-126.30312894720473, 14.29219617034159, -56.162671563152486, 49.25462702827337),geotrellis.proj4.CRS$$anon$3@41d0d1b7)
num_cols_rows = (7808,3892)
band_RDD = MapPartitionsRDD[40] at map at <console>:86
band_vec = EmptyRDD[9] at emptyRDD at <console>:44
band0 = MapPartitionsRDD[39] at map at <console>:83
band0_index = MapPartitionsRDD[34] at map at <console>:79
pattern = 2.tif
filepath = hdfs:///user/hadoop/spring-index/LastFreeze/1980.tif


hdfs:///user/hadoop/spring-index/LastFreeze/1980.tif

In [3]:
    /*
     Create a GeoTiff and save to HDFS.
    */

    val cluster_tile = DoubleArrayTile(band0.values.collect(), num_cols_rows._1, num_cols_rows._2)
    val geoTiff = SinglebandGeoTiff(cluster_tile, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT)

    //sc.parallelize(GeoTiffWriter.write(geoTiff)).saveAsObjectFile("hdfs:///users/emma/spring-index/BloomFinal/clusters_3.tif")
    GeoTiffWriter.write(geoTiff, "/tmp/test3.tif")

Name: Unknown Error
Message: lastException: Throwable = null
<console>:42: error: not found: value band0
           val cluster_tile = DoubleArrayTile(band0.values.collect(), num_cols_rows._1, num_cols_rows._2)
                                              ^
<console>:42: error: not found: value num_cols_rows
           val cluster_tile = DoubleArrayTile(band0.values.collect(), num_cols_rows._1, num_cols_rows._2)
                                                                      ^
<console>:42: error: not found: value num_cols_rows
           val cluster_tile = DoubleArrayTile(band0.values.collect(), num_cols_rows._1, num_cols_rows._2)
                                                                                        ^
<console>:43: error: not found: value projected_extent
           val geoTiff = SinglebandGeoTiff(cluster_tile, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT)
                                                         ^
<console>